In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os 
import pickle
import timeit
import math
import time
import sparse
import dask
import dask.array as da
from dask.diagnostics import ProgressBar
from scipy import interpolate
from pylab import *
from numpy import *
from glob import glob
from os import path
from tqdm import tqdm

In [2]:
outputDir = 'data/posterior_computation_data/'
gom_masks = xr.open_dataset(outputDir + 'domain_cell_masks.nc')

# GLOBAL CONSTANTS
MIN_LON = np.min(gom_masks['lon'].values)
MAX_LON = np.max(gom_masks['lon'].values)
MIN_LAT = np.min(gom_masks['lat'].values)
MAX_LAT = np.max(gom_masks['lat'].values)

#domain width and height (cell counts)
LAT_SIZE = gom_masks.dims['lat']
LON_SIZE = gom_masks.dims['lon']

#cell size
D_LON = gom_masks["lon"][1].values - gom_masks["lon"][0].values
D_LAT = gom_masks["lat"][1].values - gom_masks["lat"][0].values

BIN_CELL_LATS = gom_masks.bin_cell_lats.values
BIN_CELL_LONS = gom_masks.bin_cell_lons.values

MIN_LON, MAX_LON, MIN_LAT, MAX_LAT,LAT_SIZE,LON_SIZE, D_LON,D_LAT

(-97.98001098632812,
 -76.45999145507812,
 18.140000343322754,
 31.899998664855957,
 345,
 539,
 0.03997802734375,
 0.03999900817871094)

In [3]:
np.sum(gom_masks.oceanmask.values[np.where(gom_masks.source_cells_map > 0.0)])

111347.0

In [4]:
# Save domain_cell_tree
fileObj = open(outputDir + 'output_dict.obj', 'rb')
output = pickle.load(fileObj)
fileObj.close()

In [5]:
n_cell_beaching = output['n_cell_beaching']
n_cell_source = output['n_cell_source']
n_window_beaching = output['n_window_beaching'] 
n_window_source = output['n_window_source']

particle_count = output['particle_count']

beaching_cells = output['beaching_cells'] 
beaching_cell_tree = output['beaching_cell_tree']

source_cell_mask = output['source_cell_mask']
source_cells = output['source_cells']
source_cell_tree = output['source_cell_tree']

beaching_windows = output['beaching_windows']
source_windows = output['source_windows']
d = output['d']
beaching_ym_mat = output['beaching_ym_mat']
source_ym_mat = output['source_ym_mat']

n_cell_beaching,n_cell_source,n_window_beaching, n_window_source

(188, 114024, 30, 36)

In [6]:
import cartopy.io.shapereader as shpreader
from shapely.geometry import Point

# Mexico Border
mexico_border = ((gom_masks.coastalmask + ((gom_masks['lat'] <= 25.57) & (gom_masks['lon'] < -85.5))) == 2)

# Caribbean/Island Border
ci_border = ((gom_masks.coastalmask + ((gom_masks['lat'] <= 27.5) & (gom_masks['lon'] >= -85.5) & 
                                      ~((gom_masks['lat'] <= 27.5) & (gom_masks['lon'] >= -85.5) & 
                                        (gom_masks['lat'] >= 24.2) & (gom_masks['lon'] <= -79.5)))) == 2)


usa_border = (gom_masks.coastalmask  - (mexico_border + ci_border) )

all_gom_regions = np.array(['Caribbean','Mexico','Texas', 'Louisiana', 'Mississippi','Alabama', 'Florida'])
usa_lats_idx, usa_lons_idx = np.where(usa_border == 1.0)

usa_lats = usa_border['lat'].values[usa_lats_idx]
usa_lons = usa_border['lon'].values[usa_lons_idx]

usa_state_idx = np.ones(len(usa_lats))

for i1,(lon_,lat_) in tqdm(enumerate(zip(usa_lons,usa_lats))):
    temp_dist = np.inf
    shpfilename = shpreader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_1_states_provinces')
    reader = shpreader.Reader(shpfilename)
    states = reader.records()
    for state in states:
        geom = state.geometry
        state_name = str(state.attributes['name_en']).replace('\x00','')
        
        if state_name in all_gom_regions:
            dist = geom.distance(Point(lon_, lat_))
            if dist < temp_dist:
                idx_state = np.where(all_gom_regions == state_name)[0]
                temp_dist = dist
    usa_state_idx[i1] += idx_state

1479it [00:18, 81.63it/s]


Assign Source Region Groups

In [7]:
state_ids = np.zeros((LAT_SIZE, LON_SIZE))
state_ids[usa_lats_idx, usa_lons_idx] = usa_state_idx

mbv = np.array(mexico_border.values, dtype = np.float32)
mbv[np.where(mexico_border == 1.0)] = 2.0

all_coastal_regions = (state_ids + mbv + ci_border)

In [8]:
all_gom_regions_o = ['LaTex Shelf', 'nWFS Shelf', 'sWFS Shelf', 'CCarribean', 'Campeche Bay', 'weGOM', 'eeGOM']
o_regions = np.zeros((len(all_gom_regions_o), LAT_SIZE, LON_SIZE))

last_region_idx = np.max(all_coastal_regions).values
gom_cutout = (gom_masks.landmask + gom_masks.coastalmask).astype('bool')

# LaTex Shelf
latex_self = ((~(gom_cutout) & (gom_masks.lon < -90.0) & (gom_masks.lat > 27.0)) 
              & ~(~(gom_cutout)& (gom_masks.depth > 500) & (gom_masks.lon < -90.0) & (gom_masks.lat > 27.0)))
o_regions[0, :, :] += latex_self

# nWFS Shelf
nWFS_shelf = (~(gom_cutout) & (gom_masks.lon >= -90.0) & (gom_masks.lat > 29.0) & (gom_masks.lon < -82.5))
o_regions[1, :, :] += nWFS_shelf

# sWFS Shelf
sWFS_shelf_1 = (~(~(gom_cutout) & (gom_masks.depth >= 200) & (gom_masks.lon > -85.5) & (gom_masks.lon < -81.0) & (gom_masks.lat <= 29.0) & (gom_masks.lat > 24.0)))
sWFS_shelf = sWFS_shelf_1 & ((~(gom_cutout) & (gom_masks.lon > -85.5) & (gom_masks.lon < -81.0) & (gom_masks.lat <= 29.0) & (gom_masks.lat > 24.0)))
o_regions[2, :, :] += sWFS_shelf

# Carribean
carribean = (~(gom_cutout) & (gom_masks.lon > -85.5) & (np.power((gom_masks.lat - 20.0) / .9, 2) + (np.power((gom_masks.lon + 80.) / 1.2, 2)) < 22) )
o_regions[3, :, :] += carribean

# Campeche Bay
campeche_bay = (~(gom_cutout)  & (np.power((gom_masks.lat - 17.0) , 2) + (np.power((gom_masks.lon + 95.) / 1.5, 2)) < 22))
o_regions[4, :, :] += campeche_bay

# weGOM
weGOM = (((gom_masks.lat) + 2.*(gom_masks.lon + 90) < 27) & ((gom_masks.lat) -1.5*(gom_masks.lon + 86) > 24) 
         & ~(gom_cutout) 
         & ~((~(gom_cutout)  & (np.power((gom_masks.lat - 17.0) , 2) + (np.power((gom_masks.lon + 95.) / 1.5, 2)) < 22)))
         & ~((~(gom_cutout) & (gom_masks.lon < -90.0) & (gom_masks.lat > 27.0)) & ~(~(gom_cutout)& (gom_masks.depth > 500) & (gom_masks.lon < -90.0) & (gom_masks.lat > 27.0))))
o_regions[5, :, :] += weGOM

# eeGOM
eeGOM = (~(((gom_masks.lat) + 2.*(gom_masks.lon + 90) < 27) & ((gom_masks.lat) -1.5*(gom_masks.lon + 86) > 24)) & ~(gom_cutout)
         & ~(~(gom_cutout) & (gom_masks.lon > -85.5) & (np.power((gom_masks.lat - 20.0) / .9, 2) + (np.power((gom_masks.lon + 80.) / 1.2, 2)) < 22) )
         & ~ (sWFS_shelf_1 & ((~(gom_cutout) & (gom_masks.lon > -85.5) & (gom_masks.lon < -81.0) & (gom_masks.lat <= 29.0) & (gom_masks.lat > 24.0))))
         & ~(~(gom_cutout) & (gom_masks.lon >= -90.0) & (gom_masks.lat > 29.0) & (gom_masks.lon < -82.5))
         & ~((~(gom_cutout) & (gom_masks.lon < -90.0) & (gom_masks.lat > 27.0)) & ~(~(gom_cutout)& (gom_masks.depth > 500) & (gom_masks.lon < -90.0) & (gom_masks.lat > 27.0))))
o_regions[6, :, :] += eeGOM

In [9]:
coastal_names = all_gom_regions
coastal_maps = all_coastal_regions

ocean_names = all_gom_regions_o
ocean_maps = o_regions

ocean_names_len = len(coastal_names)
ocean_region_ids = np.arange(ocean_names_len+1, 2*ocean_names_len + 1 )[:, None, None]

argm_temp = (ocean_maps  * ocean_region_ids)
ocean_regions_groups_map = np.sum(argm_temp, axis = 0)

##########################
group_source_region_map   = coastal_maps+ocean_regions_groups_map 
gom_masks = gom_masks.assign(group_source_region_map  = (('lat', 'lon'), group_source_region_map.values ))
##########################

Assign Beaching Region Groups

In [10]:
beaching_cell_mask = (gom_masks.nurdlecount > 0.0)
group_beaching_region_map = (group_source_region_map * beaching_cell_mask)

Assign Posterior Aggregation Labels for groups and periods

In [11]:
# Used for Cell and Window Level Aggregation
source_cell_groups = np.tile(np.arange(n_cell_source), (d+1))
beaching_cell_groups = np.tile(np.arange(n_cell_beaching), n_window_beaching)
beaching_window_groups = np.repeat(np.arange(n_window_beaching), n_cell_beaching)

# Used for Region and Month Level Aggregation
group_source_region = group_source_region_map.values[source_cell_mask]
assert(len(np.where(group_source_region == 0.0)[0]) == 0.0)

group_beaching_region = group_beaching_region_map.values[beaching_cell_mask]
assert(len(np.where(group_beaching_region == 0.0)[0]) == 0.0)

group_beaching_month = np.where(beaching_ym_mat >= 0.0)[1] + 1

group_source_region_names = np.concatenate([coastal_names, ocean_names])


In [12]:
import pickle

output = {}
output['source_cell_groups'] = source_cell_groups
output['beaching_cell_groups'] = beaching_cell_groups
output['beaching_window_groups'] = beaching_window_groups

output['group_source_region'] = group_source_region
output['group_beaching_region'] = group_beaching_region
output['group_beaching_month'] = group_beaching_month
output['group_source_region_names'] = group_source_region_names
# Save domain_cell_tree
fileObj_output = open(outputDir + 'agg_dict.obj', 'wb')
pickle.dump(output,fileObj_output)
fileObj_output.close()

In [13]:
gom_masks.to_netcdf(outputDir +'domain_cell_masks_w_regions.nc')

In [14]:
# remove zarr
# import shutil
# shutil.rmtree(outputDir + '/source_agg_n_post.zarr', ignore_errors=True)